# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Открываю файл с данными и изучаю общую информацию

In [12]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
display(df.sample(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
13235,0,-174.547196,43,среднее,1,женат / замужем,0,M,сотрудник,0,125790.812968,на покупку своего автомобиля
13332,1,-3975.201410,39,неоконченное высшее,2,женат / замужем,0,M,сотрудник,0,273634.348125,заняться образованием
7891,2,-3047.766910,48,среднее,1,женат / замужем,0,M,сотрудник,0,145705.902100,высшее образование
543,2,-1104.789371,25,среднее,1,гражданский брак,1,M,сотрудник,0,116815.893498,сыграть свадьбу
12521,0,-838.685410,41,среднее,1,гражданский брак,1,F,компаньон,0,179069.576582,на покупку автомобиля
15271,1,-2520.089391,47,среднее,1,в разводе,3,M,сотрудник,1,304343.701944,операции с коммерческой недвижимостью
9571,0,-2005.219883,50,среднее,1,гражданский брак,1,F,сотрудник,0,106993.072745,на проведение свадьбы
17021,0,-4599.601996,35,среднее,1,женат / замужем,0,F,сотрудник,0,157998.993928,покупка своего жилья
19181,0,-3155.974871,45,среднее,1,женат / замужем,0,F,сотрудник,0,212739.148176,строительство недвижимости
16872,1,-101.861350,28,высшее,0,гражданский брак,1,M,сотрудник,0,101948.053221,сыграть свадьбу


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [14]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [15]:
 pd.DataFrame(round((df.isna().mean()*100),2)).style.background_gradient('coolwarm') 


,0
children,0
days_employed,10.1
dob_years,0
education,0
education_id,0
family_status,0
family_status_id,0
gender,0
income_type,0
debt,0


In [17]:
def df_info(data):
    print('----------------------------------------')
    print(data.info())
    print('----------------------------------------')
    display(pd.DataFrame(round((df.isna().mean()*100),2)).style.background_gradient('coolwarm'))
    print('----------------------------------------')

In [18]:
df_info(df)

----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None
----------------------------------------


,0
children,0
days_employed,10.1
dob_years,0
education,0
education_id,0
family_status,0
family_status_id,0
gender,0
income_type,0
debt,0


----------------------------------------


**Вывод**

В этом разделе я посмотрел сначала 15 случайных значений таблицы, далее посмотрел общую информацию о таблице и после 
посмотрел количество пропущенных значений в процентах, и где они находятся.

Пропущенных значений почти 10% в графе стаж(days_employed) и ежемесячный доход(total_income) и их одинаковое количество ....хмммм...что бы это значило..

И неучесть эти значения мы не можем, так как 10% могут сильно повлиять на конечный результат.
Предположим, что это самозанятые граждани без декларированных доходов (так как их возраст говорит о том, что стаж у них должен быть)

Так, взвесив все за и против, мы всё таки можем поставить и оставить пропуски в days_employed, ежемесячный доход мы усредним, 
но при этом необходимо сообщить Заказчику о возникшей проблеме(то есть Вам) и запросить пояснение по этим строкам, чтобы выдать корректный результат

## Шаг 2. Предобработка данных

### Обработка пропусков

In [19]:
df.loc[df['total_income'].isna(), 'total_income'].head() 

12   NaN
26   NaN
29   NaN
41   NaN
55   NaN
Name: total_income, dtype: float64

In [20]:
df_temp = df.copy() 

In [21]:
df_temp.loc[df['total_income'].isna(), 'total_income'] = df['total_income'].median() 
df_temp.loc[df['total_income'].isna(), 'total_income'].head()

12    145017.937533
26    145017.937533
29    145017.937533
41    145017.937533
55    145017.937533
Name: total_income, dtype: float64

In [22]:
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [23]:
for type_unique in df['income_type'].unique(): 
    display(type_unique)
    

'сотрудник'

'пенсионер'

'компаньон'

'госслужащий'

'безработный'

'предприниматель'

'студент'

'в декрете'

In [24]:
def no_nan_univ(dataframe, category, value):
    for i in dataframe[category].unique():
        dataframe.loc[(dataframe[category]== i) & (dataframe[value].isna()), value] = \
        dataframe.loc[dataframe[category]== i, value].median()
    return dataframe
df = pd.read_csv('/datasets/data.csv')
df_notnan = no_nan_univ(df, 'income_type', 'total_income')
df_info(df_notnan)


----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None
----------------------------------------


,0
children,0
days_employed,10.1
dob_years,0
education,0
education_id,0
family_status,0
family_status_id,0
gender,0
income_type,0
debt,0


----------------------------------------


In [25]:
df_notnan['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [26]:
df.loc[df['income_type'] == 'пенсионер','days_employed'] = df.loc[df['income_type'] == 'пенсионер','days_employed'] //24 
df['education'] = df['education'].str.lower()
df['gender'] = df['gender'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['days_employed'] = df['days_employed'].abs()

display(df.sample(15))
 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16745,0,16238.000000,59,среднее,1,в разводе,3,f,пенсионер,0,85477.663747,дополнительное образование
6014,0,507.403092,46,среднее,1,женат / замужем,0,f,сотрудник,0,73986.292718,строительство жилой недвижимости
14505,0,1000.986893,53,высшее,0,не женат / не замужем,4,f,сотрудник,0,184331.290935,строительство недвижимости
12532,3,1375.766246,29,среднее,1,женат / замужем,0,f,сотрудник,1,181158.423789,ремонт жилью
14055,0,14455.000000,67,среднее,1,женат / замужем,0,f,пенсионер,0,53272.542218,строительство жилой недвижимости
6881,0,829.632219,28,среднее,1,не женат / не замужем,4,m,сотрудник,1,106568.584137,высшее образование
10596,0,1667.140839,29,высшее,0,женат / замужем,0,m,сотрудник,0,408179.974300,покупка жилья
2012,1,15825.000000,57,среднее,1,вдовец / вдова,2,f,пенсионер,0,228300.326500,на покупку автомобиля
4945,0,NaN,23,среднее,1,не женат / не замужем,4,m,компаньон,0,172357.950966,операции с коммерческой недвижимостью
1447,0,1980.262882,29,среднее,1,женат / замужем,0,m,сотрудник,0,117590.107671,приобретение автомобиля


In [9]:
df.query('children != -1 & children !=20').groupby('children')[['debt']].sum()

,debt
children,
0,1063
1,444
2,194
3,27
4,4
5,0


In [8]:
df_gr_family_status = df.groupby('family_status', as_index = False)[['debt']].sum()
df_gr_family_status 


,family_status,debt
0,Не женат / не замужем,274
1,в разводе,85
2,вдовец / вдова,63
3,гражданский брак,388
4,женат / замужем,931


In [30]:
df['days_employed'].fillna(0, inplace=True)

**Вывод**

<div class="alert alert-info">
    
В данном разделе я посмотрел строки,где есть пропуски по столбцу total_income,
Далее создал копию для дальнейшей работы,
После поменял пропущенные значения на медианные
И вместо прописывания напрямую условия, прописал в условие переменную из цикла, где всё будет прорабатываться по очереди.
Затем написал функцию для автоматизации

Мы заменили пропущенные значения в статье дохода на медианные. 
Стаж при этом оставили, так как для общего исследования он нам не пригодится 

Я применил метод группировки по двум столбцам debt и children для сравнения этих показателей,
исключая аномалии в виде -1 и 20 детей.
Применил метод группировки по двум столбцам debt и family_status для сравнения этих показателей.
Применил метод группировки по двум столбцам debt и new_purpose для сравнения этих показателей.
Так же я заменил все значения на строчные буквы, избавился от отрицательных значений и разобрался со стажем, поделив на 24часа, так как у пенсионеров данные предположительно были предоставлены в минутах

Кроме этого, в рамках второй корректировки применил для столбца 'days_employed' метод fillna(), чтобы избавиться от Nan-ов 

### Замена типа данных

In [28]:
df ['days_employed'] = df['days_employed'].astype('int')
df ['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [20]:
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Перевёл значения из float в int в столбцах 'days_employed' и 'total_income'

### Обработка дубликатов

In [21]:
print(df.duplicated().sum()) 


71


In [22]:
df = df.drop_duplicates().reset_index(drop=True) 

In [23]:
print(df.duplicated().sum()) 

0


**Вывод**

Можно пренебречь этой цифрой, так как их очень мало в общем количестве имеющихся данных.  Сохранил в переменной df таблицу, очищенную от дубликатов, с новой индексацией. Продолжаю двигаться дальше

### Лемматизация

In [65]:
for type_unique in df['purpose'].unique():  
    display(type_unique)

'покупка жилья'

'приобретение автомобиля'

'дополнительное образование'

'сыграть свадьбу'

'операции с жильем'

...

'высшее образование'

'покупка жилья для сдачи'

'на покупку автомобиля'

'ремонт жилью'

'заняться высшим образованием'

<div class="alert alert-info">
    Посмотрел, какие вообще у нас есть категории в столбце purpose

In [25]:
from pymystem3 import Mystem
m = Mystem()

def lemmatize_purpose(string):                             
    lemma = m.lemmatize(string)
    return lemma

df['purpose'] = df['purpose'].apply(lemmatize_purpose)

dest_str = ['жилье', 'автомобиль', 'образование', 'недвижимость', 'свадьба'] 

def filter_purpose(str_lem):
    """
    создаём функцию, которая отсеивает ненужные слова, возвращая только основные понятия
    """
    for filtr in dest_str:                                 
        if filtr in str_lem:                               
            return filtr                                   

df['new_purpose'] = df['purpose'].apply(filter_purpose)

df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,"[покупка, , жилье, \n]",жилье
1,1,4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,"[приобретение, , автомобиль, \n]",автомобиль
2,0,5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,"[покупка, , жилье, \n]",жилье
3,3,4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,"[дополнительный, , образование, \n]",образование
4,0,14177.000000,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,"[сыграть, , свадьба, \n]",свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529.316663,43,среднее,1,гражданский брак,1,f,компаньон,0,224791.862382,"[операция, , с, , жилье, \n]",жилье
21450,0,14330.000000,67,среднее,1,женат / замужем,0,f,пенсионер,0,155999.806512,"[сделка, , с, , автомобиль, \n]",автомобиль
21451,1,2113.346888,38,среднее,1,гражданский брак,1,m,сотрудник,1,89672.561153,"[недвижимость, \n]",недвижимость
21452,3,3112.481705,38,среднее,1,женат / замужем,0,m,сотрудник,1,244093.050500,"[на, , покупка, , свой, , автомобиль, \n]",автомобиль


In [26]:
purposes_keys = {'жилье', 'недвижимость', 'автомобиль', 'образование', 'свадьба'}

def get_purpose(data):
    
    """Присваивает строке категорию цели"""
    
    intersection = list(purposes_keys & set(m.lemmatize(data['purpose'])))
    
    if not intersection:
        return 'категория не определена'
    return intersection[0]

df_example = pd.read_csv('/datasets/data.csv').head()

df_example.apply(get_purpose, axis=1)

0          жилье
1     автомобиль
2          жилье
3    образование
4        свадьба
dtype: object

**Вывод**

<div class="alert alert-info">
    Создал функцию для лемматизации отдельной строки, перебрал с помощью  filtr каждое понятие и сравнил со строкой из таблицы,
 в случае успеха функция возвращает основное понятие.
 
 Добавил новый столбец new_purpose, который обозначает цель кредита одним словом после лемматизации. Так удобнее будет    группировать и сравнивать цели и просрочку по кредитам

### Категоризация данных

In [32]:
income_quantile = df['total_income'].quantile([0.2, 0.4, 0.6, 0.8, 0.99]).round().to_dict()
income_quantile

{0.2: 98662.0, 0.4: 132142.0, 0.6: 161151.0, 0.8: 214270.0, 0.99: 505068.0}

In [45]:
def income_category(value):
    for k, v in income_quantile.items():
        if value <= v:
            return k
        
df['income_category'] = df['total_income'].apply(income_category)
df.sample(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_category
11488,0,13845,66,среднее,1,вдовец / вдова,2,f,пенсионер,1,55238,сделка с автомобилем,0.20
10882,1,602,42,среднее,1,гражданский брак,1,f,сотрудник,0,136692,сыграть свадьбу,0.60
20910,0,2749,27,высшее,0,не женат / не замужем,4,m,сотрудник,0,272065,сделка с подержанным автомобилем,0.99
16445,2,724,41,неоконченное высшее,2,женат / замужем,0,m,сотрудник,0,139568,строительство недвижимости,0.60
17285,2,101,29,среднее,1,женат / замужем,0,m,госслужащий,0,120052,получение дополнительного образования,0.40
4813,0,6199,55,среднее,1,женат / замужем,0,f,сотрудник,0,121464,покупка коммерческой недвижимости,0.40
5985,0,1693,35,высшее,0,женат / замужем,0,f,компаньон,0,220955,покупка жилья для семьи,0.99
16274,0,16593,50,среднее,1,женат / замужем,0,f,пенсионер,0,166602,на покупку подержанного автомобиля,0.80
18124,1,1632,32,среднее,1,в разводе,3,f,сотрудник,0,124401,автомобиль,0.40
10054,0,1133,48,среднее,1,гражданский брак,1,f,сотрудник,0,62916,свадьба,0.20


In [50]:
df['income_category'].fillna(0, inplace=True)

In [55]:
df['income_category'].unique()

array([4, 1, 2, 0, 3])

<div class="alert alert-info">
    Сверху я убрал Nan-ы в графе 'income_category', а снизу воспользовался рекомендациями по методу qcut

In [56]:
df.describe(percentiles=[0, 1/3, 2/3, 1])

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income,income_category
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04,21525.000000
mean,0.538908,4205.018815,43.293380,0.817236,0.972544,0.080883,1.652248e+05,2.000000
std,1.381587,6324.752078,12.574584,0.548138,1.420324,0.272661,9.804367e+04,1.414246
min,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.066700e+04,0.000000
0%,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.066700e+04,0.000000
33.3%,0.000000,951.000000,36.000000,1.000000,0.000000,0.000000,1.192457e+05,1.000000
50%,0.000000,1808.000000,42.000000,1.000000,0.000000,0.000000,1.425940e+05,2.000000
66.7%,1.000000,3279.000000,50.000000,1.000000,1.000000,0.000000,1.723570e+05,3.000000
100%,20.000000,395302.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06,4.000000
max,20.000000,395302.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06,4.000000


In [54]:
df['income_category'] = pd.qcut(df['total_income'],
                              q=[0, .2, .4, .6, .8, 1],
                              labels=False,
                              precision=0)
df.sample(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_category
20741,0,11283,54,среднее,1,женат / замужем,0,f,госслужащий,0,216454,на покупку подержанного автомобиля,4
12204,2,5775,35,среднее,1,женат / замужем,0,m,сотрудник,0,140235,на покупку подержанного автомобиля,2
2103,1,1481,30,среднее,1,женат / замужем,0,f,компаньон,0,85813,строительство недвижимости,0
2213,0,976,32,среднее,1,женат / замужем,0,f,сотрудник,0,207597,ремонт жилью,3
18471,1,5707,39,высшее,0,гражданский брак,1,f,госслужащий,0,141770,на проведение свадьбы,2
5970,1,4635,35,среднее,1,женат / замужем,0,f,сотрудник,0,72772,автомобили,0
21362,0,873,27,высшее,0,женат / замужем,0,f,компаньон,0,165224,приобретение автомобиля,3
16104,0,1182,33,среднее,1,женат / замужем,0,f,сотрудник,0,173813,покупка своего жилья,3
11003,0,451,23,среднее,1,женат / замужем,0,f,сотрудник,0,74012,дополнительное образование,0
7822,0,1190,30,неоконченное высшее,2,не женат / не замужем,4,m,сотрудник,0,153523,сделка с подержанным автомобилем,2


In [29]:
median_1 = df.loc[df['income_type'] == 'сотрудник', 'total_income'].median()
median_1

142594.39684740017

In [30]:
df_gr_income_type = df.groupby(['income_type']).agg({'debt': 'sum', 'total_income': ['mean','median']}) 
df_gr_income_type 


debt   total_income               
                  sum           mean         median
income_type                                        
безработный         1  131339.751676  131339.751676
в декрете           1   53829.130729   53829.130729
госслужащий        86  168863.097622  150447.935283
компаньон         376  199451.763400  172357.950966
пенсионер         216  135251.098492  118514.486412
предприниматель     0  499163.144947  499163.144947
сотрудник        1061  159566.756870  142594.396847
студент             0   98201.625314   98201.625314

In [31]:
df_gr_new_purpose = df.groupby('new_purpose', as_index = False)[['debt']].sum()
df_gr_new_purpose


,new_purpose,debt
0,автомобиль,403
1,жилье,308
2,недвижимость,474
3,образование,370
4,свадьба,186


**Вывод**

<div class="alert alert-info">
    Автоматизировал процесс категоризации для дохода. Длч этого применил квантили) Поместил результат в словарь, 
где в ключах будет категория а в значениях - максимальный заработок в данной категории.

Самый большой показатель должников у сотрудников с медианным доходом по списку,
у пенсионеров при этом меньше (в нижней границе по медиане), 
как и у компаньонов (в верхнй границе по медиане). 
Пенсионеры, компаньоны и сотрудники большинство в этой таблице
Применил метод группировки по трём столбцам income_type, debt и total_income для сравнения этих показателей. 
Определение средней и медианной зарплаты этих пользователей и соотношение их категории income_type 
поможет более точно понять зависимость total_income и debt 

## Шаг 3. вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [32]:
df.query('children !=-1 & children !=20 & gender != "XNA"').pivot_table(index = 'children', 
                                                                             values = 'debt', aggfunc = lambda x: '{:.2%}'.format(x.mean()))

,debt
children,
0,7.54%
1,9.23%
2,9.45%
3,8.18%
4,9.76%
5,0.00%


In [33]:
df.query('children !=-1 & children !=20 & gender != "XNA"').pivot_table(index = 'children', 
                                                                             values = 'debt', aggfunc = 'mean')

,debt
children,
0,0.075438
1,0.092346
2,0.094542
3,0.081818
4,0.097561
5,0.000000


**Вывод**

<div class="alert alert-info">
    ОТВЕТ: Доля пользователей, у которых четверо детей оказалась наиболее высокая относительно количества возвратов кредита в срок. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [34]:
df.pivot_table(index = 'family_status_id', 
               values = 'debt', aggfunc = lambda x: '{:.2%}'.format(x.mean()))

,debt
family_status_id,
0,7.55%
1,9.35%
2,6.57%
3,7.11%
4,9.75%


**Вывод**

<div class="alert alert-info">
    ОТВЕТ: Люди состоящие в браке, как люди не состоящие в браке, чаще остальных не возвращали кредит в срок

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [64]:
df.pivot_table(index = 'income_category', 
               values = 'debt', aggfunc = lambda x: '{:.1%}'.format(x.mean()))

,debt
income_category,
0,8.0%
1,8.4%
2,8.7%
3,8.4%
4,7.0%


**Вывод**

<div class="alert alert-info">
    ОТВЕТ: Доход не влияет на возврат кредита в срок

- Как разные цели кредита влияют на его возврат в срок?

In [36]:
df.pivot_table(index = 'new_purpose', 
               values = 'debt', aggfunc = 'mean')

,debt
new_purpose,
автомобиль,0.093590
жилье,0.069058
недвижимость,0.074634
образование,0.092200
свадьба,0.080034


**Вывод**

<div class="alert alert-info">
    ОТВЕТ: Пользователи, которые брали кредит на автомобиль чаще не возвращали кредит в срок. 

## Шаг 4. Общий вывод

<div class="alert alert-info">
   Проанализировав таблицу, могу сказать следующее:

Должниками по кредиту чаще всего становятся люди:

1)в должности "сотрудник" с зарплатой менее 161 тысячи

2)с 4 детьми состоящие в браке 

3)люди, которые берут деньги на покупку автомобиля

4)люди, не состоящие в браке и относящиеся к категориям выше, так же в группе риска

5)Самый большой показатель должников у "сотрудников" с медианным доходом 145 тысяч по списку,у "пенсионеров" при этом меньше (в нижней границе по медиане), как и у "компаньонов" (в верхней границе по медиане).
 
Рекомендации:
Таблица представлена с пропусками в данных,с разными регистрами, а так же  с аномальными значениями в столбцах "children" и "days_employed". 

Возможные причины пропусков в данных в стобцах "total_income" и "days_employed":
1)Человеческий фактор
2)Самозанятые граждани без декларированных доходов
3)Технические ошибки в программе при формировании таблицы(совместимость данных)
